# Residential Diversity Map

## 1. Define Styling
Description

Example: https://mkz.smartuse.ch/uploads/assets/views/SU-002-Wohnungsdiversitaet.html

**Columns:**  
        *Size*: BTOT from statpop  
        *Color categories*: WT1, WT2, WT3, WT4, WT5, WT6 from gws

## 2.Cookbook

### **Spatialite Querying**  

INPUT: requested data through spatial querying  
CODE: *(see below)*  
OUTPUT: <result\>.csv  

```sqlite
SELECT load_extension('mod_spatialite');

-- QUERY WT{1,2,3,4,5,6} FROM GWS
-- JOIN TABLES ON RELI
-- SPATIAL QUERY FOR SOME AREA
.header on
.mode csv
.once result3.csv
SELECT A.BTOT, A.yr, A.RELI, AsText(ST_Transform(A.geometryLV95, 4326)) AS wkt, 
B.WT1, B.WT2, B.WT3, B.WT4, B.WT5, B.WT6 from gstatpop as A 
INNER JOIN ggws as B 
ON A.RELI == B.RELI   
WHERE (A.yr=='2014' AND B.yr = '2014' AND Within(A.geometryLV95,Buffer(GeomFromText('Point(2683268 1248001)'), 1000)));
```

```python
>>> import pandas as pd
>>> df = pd.read_csv('result.csv')
df.dropna(inplace=True)
>>> df.to_csv('clear_result.csv', index=False)
```

### **Datapackage pipelines**

INPUT: 'clear_result.csv (from Spatialite Querying) + data_to_feed.yaml  
CODE: *(see below)*   
OUTPUT: datapackage.json, modified_result.csv  
  
Task: We want to give a style to the column with the highest number of apartments.
Analysis in pandas:

```python
>>> data = pd.read_csv("clear_result.csv")
>>> data[['WT1', 'WT2', 'WT3', 'WT4', 'WT5', 'WT6']].max(axis=1)
0       24
1       17
2       38
3       52
4       20
        ..
4061     1
4062    37
4063     8
4064    50
4065     6
Length: 4066, dtype: int64
>>> data[['WT1', 'WT2', 'WT3', 'WT4', 'WT5', 'WT6']].idxmax(axis=1)
0       WT3
1       WT4
2       WT2
3       WT3
4       WT5
       ... 
4061    WT3
4062    WT3
4063    WT4
4064    WT2
4065    WT2

```
This I need to rewrite to dataflows.
 
First one is through the run not flows:
```yaml
run: add_computed_field
parameters:
  resources: result
  fields:
    -
      operation: max
      target: max_number
      source:
        - WT1
        - WT2
        - WT3
        - WT4
        - WT5
        - WT6
```

CODE:

flows.apartments:

```python
from dataflows import Flow
import math

def flow(parameters, datapackage, resources, stats):

    def apartment():
        def step(row):
            if row['max_number'] is not None:
                if row['WT1'] == row['max_number']
                    row['Category'] = 'WT1'   
                elif row['WT2'] == row['max_number']
                    row['Category'] = 'WT2'
                elif row['WT3'] == row['max_number']
                    row['Category'] = 'WT3'
                elif row['WT4'] == row['max_number']
                    row['Category'] = 'WT4'
                elif row['WT5'] == row['max_number']
                    row['Category'] = 'WT5'
                else:
                    row['Category'] = 'WT6'
        return step

    return Flow(
        add_field('Category', 'string'),
        apartment()
    )
```

flows.styling:

```python
from dataflows import Flow, add_field

from util import *
import math

default_dict = getting_dictionary('template/default_dict.txt')

def flow(parameters, datapackage, resources, stats):

    def category():
        def step(row):
            if (row['Category'] == 'WT1'):
                for k,v in default_dict[0].items():
                    row[k] = v
            if (row['Category'] == 'WT2'):
                for k,v in default_dict[1].items():
                    row[k] = v
            if (row['Category'] == 'WT3'):
                for k,v in default_dict[2].items():
                    row[k] = v
            if (row['Category'] == 'WT4'):
                for k,v in default_dict[3].items():
                    row[k] = v
            if (row['Category'] == 'WT5'):
                for k,v in default_dict[4].items():
                    row[k] = v
            else:
                for k,v in default_dict[5].items():
                    row[k] = v
        return step

    return Flow(
        add_field('fill', 'string'),
        add_field('fillColor', 'string'),
        add_field('fillOpacity', 'string'),
        add_field('stroke', 'string'),
        add_field('color', 'string'),
        add_field('opacity', 'string'),
        add_field('weight', 'string'),
        add_field('radius', 'number'),
        category(),
        )
```

### **Exporter**

INPUT: datapackage.json, modified_result.csv  
CODE: *(see below)*   
OUTPUT: snapshot.json    

CODE:
```python
import yaml
import json
import geopandas as gpd

gdf = gpd.read_file('output/residential/residential.csv')
gdf.to_file("output/residential/preview.geojson", driver='GeoJSON')

with open("output/residential/datapackage.json", 'r') as j, \
     open("output/residential/preview.geojson", 'r') as l, \
     open("output/residential/snapshot.json", 'w') as r:
    data = json.load(j)
    feed = json.load(l)
    data["resources"] = [{"name": "data-layer", "mediatype": "application/vnd.simplestyle-extended", "data": {"name": "data", "type": "FeatureCollection", "features": []}},{"name": "mapbox-background", "path": "mapbox://styles/gemeindescan/ck6rp249516tg1iqkmt48o4pz", "mediatype": "application/vnd.mapbox-vector-tile", "data":"" }]
    data['resources'][0]['data']['features'] = feed['features']
    json.dump(data, r)
```